### Deploying accurate information based on Cenko 2009

I converted the apparent mags in the Cenko catalog to absolute using the usual formula: M = m - mu - K(z), where mu is the distance modulus from redshift. 
I used Planck18 cosmology for that. The K-correction accounts for the fact that we're observing redshifted light, so the filter used isn't probing the same rest-frame wavelength across redshifts. Assumed a typical GRB afterglow spectrum with beta = 0.75 for the correction. Also calculated abs mag without the K-correction to show the difference. 
Cenko compares rest-frame luminosities in the paper so this matters.

In [1]:
import pandas as pd
import numpy as np
from astropy.cosmology import Planck18 as cosmo
from astropy import units as u
import requests
from io import StringIO

In [96]:
# Step 1: Download the file
url = "https://content.cld.iop.org/journals/0004-637X/693/2/1484/revision1/apj299693t1_mrt.txt"
response = requests.get(url)
lines = response.text.splitlines()

# Step 2: Extract only data lines
data_lines = [line for line in lines if line.startswith("GRB")]
data_str = "\n".join(data_lines)

# Step 3: Define column specs and names (just the ones you want)
colspecs = [
    (0, 10),   # GRB name
    (30, 40),  # Time since burst [s]
    (41, 45),  # Filter
    (53, 54),  # Limit flag
    (54, 59),  # Magnitude
    (60, 64),  # Magnitude error
    (65, 66),  # Mag reference
]

colnames = ["GRB", "Time_sec", "Filter", "LimitFlag", "App_Mag", "Mag_Err", "Mag_Ref"]

# Step 4: Read into DataFrame
df = pd.read_fwf(StringIO(data_str), colspecs=colspecs, names=colnames)

# Step 5: Show preview
print(df.head(30))

print("--------")
unique_grbs = df['GRB'].nunique()
print(f"Number of unique GRBs in Table 1: {unique_grbs}")

print("--------")
unique_grbs_ordered = df['GRB'].drop_duplicates()
print("Unique GRBs in Table 1 (in order of appearance):")
for grb in unique_grbs_ordered:
    print(grb)




           GRB  Time_sec Filter LimitFlag  App_Mag  Mag_Err  Mag_Ref
0    GRB050412     391.0   R_C_         >    20.80      NaN        1
1    GRB050412     684.7   R_C_         >    21.40      NaN        1
2    GRB050412    1063.7      i         >    21.00      NaN        1
3    GRB050412    1646.0      z         >    20.20      NaN        1
4    GRB050412    2858.8   R_C_         >    22.50      NaN        1
5    GRB050412    2817.0      i         >    22.00      NaN        1
6   GRB050416A     216.0   I_C_       NaN    18.82     0.11        2
7   GRB050416A     371.5   I_C_       NaN    18.86     0.11        2
8   GRB050416A     527.0   I_C_       NaN    19.16     0.13        2
9   GRB050416A     691.2   I_C_       NaN    19.35     0.23        2
10  GRB050416A     846.7   I_C_       NaN    19.01     0.12        2
11  GRB050416A    1002.2   I_C_       NaN    19.53     0.18        2
12  GRB050416A    1347.8      z       NaN    19.16     0.27        2
13  GRB050416A    1874.9   I_C_   

In [115]:
# Paste the data string for Table 3 here (just GRB name and Redshift column)
data = """
050412	No	No	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<0.49  	 ...
050416A	Yes	 ... 	0.6535	0.23 +or- 0.08	0.9^+2.0_-0.3	11.6^+66.6_-8.9	2.6 +or- 1.8	1.33 (7)	0.35	 ...
050607	No	OT	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<0.72  	 ...
050713A	Yes	 ... 	 ... 	0.62^+0.12_-0.11	 ... 	 ... 	 ... 	0.72 (7)	0.31	 ...
050820A^f	Yes	 ... 	2.615	 ... 	 ... 	 ... 	 ... 	 ... 	0.40	<0.10
050908	Yes	 ... 	3.35	0.69 +or- 0.05	 ... 	 ... 	-0.4 +or- 1.1	 0.75 (12)	0.91	 ...
050915A	No	IRT	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<0.44  	 ...
060110	Yes	 ... 	 ... 	0.92^+0.30_-0.25	 ... 	 ... 	 ... 	0.23 (8)	0.80	 ...
060210	Yes	 ... 	3.91	0.93 +or- 0.06	 ... 	 ... 	7.2 +or- 0.7	 1.43 (21)	0.37	1.21^+0.16_-0.12
060502A	Yes	 ... 	1.51	0.49 +or- 0.05	 ... 	 ... 	2.1 +or- 0.3	 0.42 (24)	0.53	0.53 +or- 0.13
060510B	Yes	 ... 	4.9	0.3 +or- 0.5	 ... 	 ... 	4.2^+1.8_-2.2	0.15 (4)	0.04	 ...
060805A	No	No	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<0.76 	 ...
060906^g	Yes	 ... 	3.685	 ... 	 ... 	 ... 	2.2 +or- 0.2	 0.22 (20)	0.88	0.20^+0.01_-0.12
060908	Yes	 ... 	2.43	1.03 +or- 0.02	 ... 	 ... 	0.46 +or- 0.06	 2.26 (52)	0.82	 ...
060923A	No	IRT	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<0.41  	 ...
061222A	No	IRT	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<0.15  	 ...
070208	Yes	 ... 	1.165	0.50 +or- 0.02	 ... 	 ... 	2.18 +or- 0.12	 1.81 (23)	0.54	0.96 +or- 0.09
070419A	Yes	 ... 	0.97	-2.7 +or- 0.6	1.04 +or- 0.04	0.53 +or- 0.02	1.5 +or- 0.2	 2.57 (17)	0.87	0.70^+0.31_-0.11
070521	No	No	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<-0.03  	 ...
071003	Yes	 ... 	1.60435	1.77 +or- 0.05	 ... 	 ... 	0.86 +or- 0.19	0.48 (8)	0.27	<0.26
071010A	Yes	 ... 	 ... 	0.29 +or- 0.19	 ... 	 ... 	1.2 +or- 0.7	0.85 (6)	0.83	 ...
071011	Yes	 ... 	 ... 	0.90 +or- 0.20	 ... 	 ... 	1.9 +or- 0.7	0.26 (6)	0.66	 ...
071020	Yes	 ... 	2.145	0.89 +or- 0.12	 ... 	 ... 	0.58 +or- 0.27	 2.16 (14)	0.52	 ...
071122	Yes	 ... 	1.14	-0.08 +or- 0.09	 ... 	 ... 	1.3 +or- 0.6	 0.36 (11)	0.64	0.58 +or- 0.05
080310	Yes	 ... 	2.43	0.03 +or- 0.03	0.69 +or- 0.07	1.83 +or- 0.02	0.97 +or- 0.06	 0.55 (24)	0.79	0.10 +or- 0.02
080319A	Yes	 ... 	 ... 	-0.9 +or- 0.4	0.80 +or- 0.07	0.160^+0.136_-0.070	2.0 +or- 0.3	 1.50 (11)	0.41	 ...
080319B	Yes	 ... 	0.937	1.93^+0.04_-0.06	1.238 +or- 0.004	10.10 +or- 0.17	0.50 +or- 0.02	  0.57 (280)	0.52	 ...
080319C	Yes	 ... 	1.95	1.4 +or- 0.2	 ... 	 ... 	2.4 +or- 0.2	2.32 (3)	0.36	0.67 +or- 0.06
080320	Yes	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	 ... 	<0.31  	 ...
"""

# Parse each line
rows = []
for line in data.strip().splitlines():
    parts = line.split("\t")
    if not re.match(r"^\d{6}[A-Z]?", parts[0]):
        continue
    name = "GRB" + parts[0].strip("^f^g")
    redshift_raw = parts[3].strip()
    try:
        redshift = float(redshift_raw)
    except:
        redshift = None
    rows.append((name, redshift))
    

# Create and preview DataFrame
df_z = pd.DataFrame(rows, columns=["GRB", "Redshift"])
print(df_z.head(30))

print("---------")
unique_grbs = df_z['GRB'].nunique()
print(f"Number of unique GRBs in Table 3: {unique_grbs}")


print("--------")
unique_grbs_ordered = df_z['GRB'].drop_duplicates()
print("Unique GRBs in Table 3 (in order of appearance):")
for grb in unique_grbs_ordered:
    print(grb)



           GRB  Redshift
0    GRB050412       NaN
1   GRB050416A   0.65350
2    GRB050607       NaN
3   GRB050713A       NaN
4   GRB050820A   2.61500
5    GRB050908   3.35000
6   GRB050915A       NaN
7    GRB060110       NaN
8    GRB060210   3.91000
9   GRB060502A   1.51000
10  GRB060510B   4.90000
11  GRB060805A       NaN
12   GRB060906   3.68500
13   GRB060908   2.43000
14  GRB060923A       NaN
15  GRB061222A       NaN
16   GRB070208   1.16500
17  GRB070419A   0.97000
18   GRB070521       NaN
19   GRB071003   1.60435
20  GRB071010A       NaN
21   GRB071011       NaN
22   GRB071020   2.14500
23   GRB071122   1.14000
24   GRB080310   2.43000
25  GRB080319A       NaN
26  GRB080319B   0.93700
27  GRB080319C   1.95000
28   GRB080320       NaN
---------
Number of unique GRBs in Table 3: 29
--------
Unique GRBs in Table 3 (in order of appearance):
GRB050412
GRB050416A
GRB050607
GRB050713A
GRB050820A
GRB050908
GRB050915A
GRB060110
GRB060210
GRB060502A
GRB060510B
GRB060805A
GRB060906
GRB06090

In [121]:
# Merge redshift info into the photometry table by GRB name
# Clean GRB names in both tables
#df["GRB"] = df["GRB"].str.strip().str.replace(r"[\\\^\s]", "", regex=True)
#df_z["GRB"] = df_z["GRB"].str.strip().str.replace(r"[\\\^\s]", "", regex=True)

merged_df = pd.merge(df, df_z, on="GRB", how="left")

# Preview the merged table
print(merged_df.head(50))

# Optional: Save to CSV
merged_df.to_csv("merged_photometry_with_redshift.csv", index=False)


           GRB  Time_sec Filter LimitFlag  App_Mag  Mag_Err  Mag_Ref  Redshift
0    GRB050412     391.0   R_C_         >    20.80      NaN        1       NaN
1    GRB050412     684.7   R_C_         >    21.40      NaN        1       NaN
2    GRB050412    1063.7      i         >    21.00      NaN        1       NaN
3    GRB050412    1646.0      z         >    20.20      NaN        1       NaN
4    GRB050412    2858.8   R_C_         >    22.50      NaN        1       NaN
5    GRB050412    2817.0      i         >    22.00      NaN        1       NaN
6   GRB050416A     216.0   I_C_       NaN    18.82     0.11        2    0.6535
7   GRB050416A     371.5   I_C_       NaN    18.86     0.11        2    0.6535
8   GRB050416A     527.0   I_C_       NaN    19.16     0.13        2    0.6535
9   GRB050416A     691.2   I_C_       NaN    19.35     0.23        2    0.6535
10  GRB050416A     846.7   I_C_       NaN    19.01     0.12        2    0.6535
11  GRB050416A    1002.2   I_C_       NaN    19.53  

In [130]:
from astropy.cosmology import Planck18 as cosmo
import numpy as np

# Assumed spectral slope
beta = 0.75

# Filter valid entries
valid = merged_df.dropna(subset=["Redshift", "App_Mag", "Time_sec"]).copy()

# Luminosity distance in pc
dl_pc = cosmo.luminosity_distance(valid["Redshift"]).to("pc").value

# Distance modulus
mu = 5 * np.log10(dl_pc / 10)

# K-correction (approximate for power-law SED)
k_corr = -2.5 * (1 - beta) * np.log10(1 + valid["Redshift"])

# Absolute magnitude without K-correction
valid["Abs_Mag_NoK"] = valid["App_Mag"] - mu

# Absolute magnitude with K-correction
valid["Abs_Mag_Kcorr"] = valid["App_Mag"] - mu - k_corr

# Rest-frame time
valid["Rest_Time_sec"] = valid["Time_sec"] / (1 + valid["Redshift"])

# Assign back to full DataFrame
for col in ["Abs_Mag_NoK", "Abs_Mag_Kcorr", "Rest_Time_sec"]:
    merged_df[col] = np.nan
    merged_df.loc[valid.index, col] = valid[col]

# Preview
print(merged_df[["GRB", "Filter", "Time_sec", "Rest_Time_sec", "App_Mag", "Redshift", "Abs_Mag_NoK", "Abs_Mag_Kcorr"]].head(20))


           GRB Filter  Time_sec  Rest_Time_sec  App_Mag  Redshift  \
0    GRB050412   R_C_     391.0            NaN    20.80       NaN   
1    GRB050412   R_C_     684.7            NaN    21.40       NaN   
2    GRB050412      i    1063.7            NaN    21.00       NaN   
3    GRB050412      z    1646.0            NaN    20.20       NaN   
4    GRB050412   R_C_    2858.8            NaN    22.50       NaN   
5    GRB050412      i    2817.0            NaN    22.00       NaN   
6   GRB050416A   I_C_     216.0     130.631993    18.82    0.6535   
7   GRB050416A   I_C_     371.5     224.674932    18.86    0.6535   
8   GRB050416A   I_C_     527.0     318.717871    19.16    0.6535   
9   GRB050416A   I_C_     691.2     418.022377    19.35    0.6535   
10  GRB050416A   I_C_     846.7     512.065316    19.01    0.6535   
11  GRB050416A   I_C_    1002.2     606.108255    19.53    0.6535   
12  GRB050416A      z    1347.8     815.119444    19.16    0.6535   
13  GRB050416A   I_C_    1874.9   